# Anti-Money Laundering System

## Problem Statement

Money Laundering is a major challenge in the present highly digitalized economic ecosystem. The fraudulent behavior of the agents aims to profit by taking control of customers accounts and try to empty the funds by transferring to another account and then cashing out of the system. The main objective of this project is to built a machine learning model based on financial transaction data for detecting such fraudulent behavior.

***CRISP-ML(Q)*** process model describes six phases:

1. Business and Data Understanding
2. Data Preparation
3. Model Building
4. Model Evaluation
5. Deployment
6. Monitoring and Maintenance

**Objective(s):** Maximizing the detection of fraud transactions via different channels.

**Constraints:** Minimizing false positives being generated for fraud transactions

# Data Collection

**Data:** This is a synthetic dataset generated using the simulator called PaySim. PaySim uses aggregated data from the private dataset to generate a synthetic dataset that resembles the normal operation of transactions and injects malicious behaviour to later evaluate the performance of fraud detection methods.

**Dataset:** 
* Our dataset has 6362620 observations about financial transactions.
* And it has 11 variables associated to each transactions. 

**Variables Description:**
* step - Maps a unit of time in the real world(1 step = 1 hour of time).

* type - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.

* amount - Amount of the transaction in local currency.

* nameOrig - Customer who started the transaction.

* oldbalanceOrg - Initial balance before the transaction.

* newbalanceOrig - New balance after the transaction.

* nameDest - Customer who is the recipient of the transaction.

* oldbalanceDest - Initial balance recipient before the transaction.

* newbalanceDest - New balance recipient after the transaction.

* isFraud - This is the transactions made by the fraudulent agents inside the simulation.

* isFlaggedFraud - The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt means to transfer more than 200.000 in a single transaction.

>  Note that there is not information for customers that start with M (Merchants).

**Required Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import sweetviz
import dtale
from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import train_test_split, learning_curve, RepeatedStratifiedKFold, cross_val_score, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz
import lightgbm as lgb
from scipy.stats import boxcox, skew, uniform, randint

**Importing the data**

In [ ]:
transaction_details_dataset = pd.read_csv("transaction_details.csv")

In [ ]:
transaction_details_dataset.isnull().values.any()

In [ ]:
print("Our dataset has {} observations and {} columns".format(transaction_details_dataset.shape[0], transaction_details_dataset.shape[1]))

In [ ]:
transaction_details_dataset.head()

In [ ]:
transaction_details_dataset.describe()

In [ ]:
transaction_details_dataset.info()

## Exploratory Data Analysis (EDA) / Descriptive Statistics

In [ ]:
print(transaction_details_dataset.type.value_counts())

In [ ]:
print('Types of fraud transactions: {}'.format(list(transaction_details_dataset.loc[transaction_details_dataset.isFraud == 1].type.drop_duplicates().values)))

fraud_transfer = transaction_details_dataset.loc[(transaction_details_dataset.isFraud == 1) & (transaction_details_dataset.type == 'TRANSFER')]
fraud_cash_out = transaction_details_dataset.loc[(transaction_details_dataset.isFraud == 1) & (transaction_details_dataset.type == 'CASH_OUT')]

print('\nNumber of fraud TRANSFER\'s: {}'.format(len(fraud_transfer)))
print('Number of fraud CASH_OUT\'s: {}'.format(len(fraud_cash_out)))

print('\nPercentage of fraud TRANSFER\'s: {} %'.format((len(fraud_transfer)/len(transaction_details_dataset)) * 100))
print('Percentage of fraud CASH_OUT\'s: {} %'.format((len(fraud_transfer)/len(transaction_details_dataset)) * 100))

In [ ]:
print('Types of transactions which are \'isFlaggedFraud\': {}'.format(list(transaction_details_dataset.loc[transaction_details_dataset.isFlaggedFraud == 1].type.drop_duplicates().values)))

transfer = transaction_details_dataset.loc[(transaction_details_dataset.type == 'TRANSFER')]
is_flagged_fraud = transaction_details_dataset.loc[(transaction_details_dataset.isFlaggedFraud == 1)]
not_flagged_fraud = transaction_details_dataset.loc[(transaction_details_dataset.isFlaggedFraud == 0)]

print('Minimum amount of transaction when \'isFlaggedFraud\' is set: {}'.format(is_flagged_fraud.amount.min()))
print('Maximum amount of transaction in a TRANSFER when \'isFlaggedFraud\' is not set: {}'.format(transfer.loc[(transfer.isFlaggedFraud == 0)].amount.max()))

In [ ]:
print('Number of TRANSFER\'s where isFlaggedFraud = 1 yet oldbalanceDest = 0, & newbalanceDest = 0: {}'.format(len(transaction_details_dataset.loc[(transaction_details_dataset.isFlaggedFraud == 1) & (transaction_details_dataset.newbalanceDest == 0) & (transaction_details_dataset.oldbalanceDest == 0)])))
print('Number of TRANSFER\'s where isFlaggedFraud = 0 yet oldbalanceDest = 0, & newbalanceDest = 0: {}'.format(len(transaction_details_dataset.loc[(transaction_details_dataset.isFlaggedFraud == 0) & (transaction_details_dataset.newbalanceDest == 0) & (transaction_details_dataset.oldbalanceDest == 0)])))

In [ ]:
print('Minimum value of newbalanceOrig when isFlaggedFraud = 0 where oldbalanceOrg = newbalanceOrig: {}'.format(transfer.loc[(transfer.isFlaggedFraud == 0) & (transfer.oldbalanceOrg == transfer.newbalanceOrig)].oldbalanceOrg.min()))
print('Maximum value of newbalanceOrig when isFlaggedFraud = 0 where oldbalanceOrg = newbalanceOrig: {}'.format(transfer.loc[(transfer.isFlaggedFraud == 0) & (transfer.oldbalanceOrg == transfer.newbalanceOrig)].oldbalanceOrg.max()))
print('Minimum value of oldbalanceOrg when isFlaggedFraud = 1: {}'.format(is_flagged_fraud.oldbalanceOrg.min()))
print('Maximum value of oldbalanceOrg when isFlaggedFraud = 1: {}'.format(is_flagged_fraud.oldbalanceOrg.max()))

In [ ]:
print('Any originator of transaction having more than 1 flagged fraud transaction? {}'.format(is_flagged_fraud.nameOrig.isin(pd.concat([not_flagged_fraud.nameOrig, not_flagged_fraud.nameDest])).any()))
print('Any destination for transaction flagged as fraud? {}'.format(is_flagged_fraud.nameDest.isin(not_flagged_fraud.nameOrig).any()))
print('Number of destination accounts flagged fraud were already destination accounts before? {}'.format(sum(is_flagged_fraud.nameDest.isin(not_flagged_fraud.nameDest))))

In [ ]:
print('Any merchants among originator account? {}'.format(transaction_details_dataset.nameOrig.str.contains('M').any()))
print('Are there any transactions having merchants as destination accounts other than \'PAYMENT\' type? {}'.format((transaction_details_dataset.loc[transaction_details_dataset.nameDest.str.contains('M')].type != 'PAYMENT').any()))
print('Any merchants among originator who accounts for \'CASH_IN\' transactions? {}'.format(transaction_details_dataset.loc[transaction_details_dataset.type == 'CASH_IN'].nameOrig.str.contains('M').any()))
print('Any merchants among originator who accounts for \'CASH_OUT\' transactions? {}'.format(transaction_details_dataset.loc[transaction_details_dataset.type == 'CASH_OUT'].nameOrig.str.contains('M').any()))

In [ ]:
not_fraud = transaction_details_dataset.loc[transaction_details_dataset.isFraud == 0]
print('Fraud TRANSFER"s where destination accounts are originator of "CASH_OUT":\n {}'.format(fraud_transfer.loc[fraud_transfer.nameDest.isin(not_fraud.loc[not_fraud.type == 'CASH_OUT'].nameOrig.drop_duplicates())]))

In [ ]:
print('Fraud TRANSFER to \'C1714931087\' occurs at step [65] whereas genuine \'CASH_OUT\' from this account occured at step = {}'.format(not_fraud.loc[(not_fraud.type == 'CASH_OUT') & (not_fraud.nameOrig == 'C1714931087')].step.values))
print('Fraud TRANSFER to \'C423543548\' occurs at step [486] whereas genuine \'CASH_OUT\' from this account occured at step = {}'.format(not_fraud.loc[(not_fraud.type == 'CASH_OUT') & (not_fraud.nameOrig == 'C423543548')].step.values))
print('Fraud TRANSFER to \'C1023330867\' occurs at step [738] whereas genuine \'CASH_OUT\' from this account occured at step = {}'.format(not_fraud.loc[(not_fraud.type == 'CASH_OUT') & (not_fraud.nameOrig == 'C1023330867')].step.values))

In [ ]:
figure, ax = plt.subplots(1, 1, figsize = (8, 6))
transaction_details_dataset.type.value_counts().plot(kind = 'bar', title = 'Transaction Type', color ='red')

In [ ]:
figure, ax = plt.subplots(1, 1, figsize = (8, 6))
ax = transaction_details_dataset.groupby(['type', 'isFraud']).size().plot(kind = 'bar')
ax.set_title('Number of actual fraud transaction per transaction type')
ax.set_xlabel('Type, isFraud')
ax.set_ylabel('Number of transactions')

for x in ax.patches:
    ax.annotate(str(format(int(x.get_height()))), (x.get_x(), x.get_height() * 1.01))

In [ ]:
figure, ax = plt.subplots(1, 1, figsize = (8, 6))
ax = transaction_details_dataset.groupby(['type', 'isFlaggedFraud']).size().plot(kind = 'bar')
ax.set_title('Number of actual fraud transaction per transaction type')
ax.set_xlabel('Type, isFlaggedFraud')
ax.set_ylabel('Number of transactions')

for x in ax.patches:
    ax.annotate(str(format(int(x.get_height()))), (x.get_x(), x.get_height() * 1.01))

In [ ]:
figure, axis = plt.subplots(2, 2, figsize = (8, 8))

figure_1 = sns.boxplot(x = 'isFlaggedFraud', y = 'amount', data = transfer, ax = axis[0][0])
axis[0][0].set_yscale('log')
figure_2 = sns.boxplot(x = 'isFlaggedFraud', y = 'oldbalanceDest', data = transfer, ax = axis[0][1])
axis[0][1].set(ylim=(0, 0.5e8))
figure_3 = sns.boxplot(x = 'isFlaggedFraud', y = 'oldbalanceOrg', data = transfer, ax = axis[1][0])
axis[1][0].set(ylim=(0, 3e7))
figure_4 = sns.regplot(x = 'oldbalanceOrg', y = 'amount', data = transfer.loc[(transaction_details_dataset.isFlaggedFraud == 1)], ax = axis[1][1])
plt.show()

## Data Preprocessing

***'TRANSFER' OR 'CASH_OUT' Transactions***

In [ ]:
transfer_or_cash_out = transaction_details_dataset.loc[(transaction_details_dataset.type == 'TRANSFER') | (transaction_details_dataset.type == 'CASH_OUT')]
np.random.seed(5)
print('We have a total of {} transactions that are either \'TRANSFER\' OR \'CASH_OUT\'.'.format(transfer_or_cash_out.shape[0]))

### Automated Library

In [ ]:
automated_report = sweetviz.analyze(transfer_or_cash_out)
automated_report.show_html('Report.html')

In [ ]:
dataset_visuals = dtale.show(transfer_or_cash_out)
dataset_visuals.open_browser()

**Cleaning Unwanted Columns**

'nameDest', 'nameOrig', 'isFlaggedFraud': These columns are of no relevance for analytically domain of work, as it does not have any nominal data. Hence, we can ignore them.

In [ ]:
transfer_or_cash_out = transfer_or_cash_out.drop(['nameDest', 'nameOrig', 'isFlaggedFraud'], axis = 1)

In [ ]:
fraud_transfer_or_cash_out = transfer_or_cash_out.loc[transfer_or_cash_out['isFraud'] == 1]
not_fraud_transfer_or_cash_out = transfer_or_cash_out.loc[transfer_or_cash_out['isFraud'] == 0]

print('Fraction of fraud transactions with \'oldbalanceDest\' = \'newbalanceDest\' after having a non-zero transaction: {}'.format(len(fraud_transfer_or_cash_out.loc[(fraud_transfer_or_cash_out.oldbalanceDest == 0) & (fraud_transfer_or_cash_out.newbalanceDest == 0) & (fraud_transfer_or_cash_out.amount)]) / len(fraud_transfer_or_cash_out)))

***Computation:***
1. Imputing null values using Mean imputation.
2. Conversion of 'categorical data' to 'numerical data' using LabelEncoder.
3. DataFrameMapper is used to map the given attribute.

In [ ]:
numerical_features = transfer_or_cash_out.select_dtypes(include = ['int32', 'int64', 'float32', 'float64']).columns
numerical_features

In [ ]:
categorical_features = transfer_or_cash_out.select_dtypes(include = ['object']).columns
categorical_features

In [ ]:
numerical_pipeline = Pipeline([('impute', SimpleImputer(strategy = 'mean'))])
categorical_pipeline = Pipeline([('label', DataFrameMapper([(categorical_features, LabelEncoder())]))])
scale_pipeline = Pipeline([('scale', MinMaxScaler())])

In [ ]:
preprocess_pipeline = ColumnTransformer([('category', categorical_pipeline, categorical_features), 
                                         ('numerical', numerical_pipeline, numerical_features)],
                                        remainder = 'passthrough')
preprocess_fit = preprocess_pipeline.fit(transfer_or_cash_out)
preprocess_fit

In [ ]:
preprocess_transform = preprocess_fit.transform(transfer_or_cash_out)
preprocess_transform

In [ ]:
preprocess_2 = pd.DataFrame(preprocess_transform, columns = [transfer_or_cash_out])
preprocess_2

## Data Visualization

In [ ]:
def correlation_plot(dataframe, labels):
    figure, axis = plt.subplots(1, 1, figsize = (8, 8))
    cmap = cm.get_cmap('inferno')
    cax = axis.imshow(dataframe.corr(), cmap = cmap)
    plt.title('Correlation HeatMap')
    axis.set_xticklabels(labels, fontsize = 12, rotation = 60)
    axis.set_yticklabels(labels, fontsize = 12)
    figure.colorbar(cax)
    plt.show()
    
correlation_plot_labels = preprocess_2.columns.tolist()

for i in range(len(correlation_plot_labels)):
    correlation_plot_labels[i] = correlation_plot_labels[i][0]
    
correlation_plot(preprocess_2, correlation_plot_labels)

In [ ]:
sns.heatmap(preprocess_2.corr())

In [ ]:
transaction_type_plot = transfer_or_cash_out.type.value_counts().plot(kind = 'bar', title = 'Transaction Type', figsize = (8, 6))

for patch in transaction_type_plot.patches:
    transaction_type_plot.annotate(str(format(int(patch.get_height()))), (patch.get_x(), patch.get_height() * 1.01))

In [ ]:
fraud_transaction_plot = pd.value_counts(transfer_or_cash_out['isFraud']).plot(kind = 'bar', title = 'Fraud Transaction', figsize = (8, 6))

for patch in fraud_transaction_plot.patches:
    fraud_transaction_plot.annotate(str(format(int(patch.get_height()))), (patch.get_x(), patch.get_height() * 1.01))

Data visualized above have quite skewed numerical variables. Therefore scaling the data to skew

In [ ]:
transfer_or_cash_out['amount_boxcox'] = preprocessing.scale(boxcox(transfer_or_cash_out['amount'] + 1)[0])
transfer_or_cash_out['amount_oldbalanceOrg'] = preprocessing.scale(boxcox(transfer_or_cash_out['oldbalanceOrg'] + 1)[0])
transfer_or_cash_out['amount_newbalanceOrig'] = preprocessing.scale(boxcox(transfer_or_cash_out['newbalanceOrig'] + 1)[0])
transfer_or_cash_out['amount_oldbalanceDest'] = preprocessing.scale(boxcox(transfer_or_cash_out['oldbalanceDest'] + 1)[0])
transfer_or_cash_out['amount_newbalanceDest'] = preprocessing.scale(boxcox(transfer_or_cash_out['newbalanceDest'] + 1)[0])

In [ ]:
figure, axis = plt.subplots(1, 3, figsize = (12, 5))

axis[0].hist(transfer_or_cash_out['amount'])
axis[0].set_xlabel('Transaction Amount')
axis[0].set_title('Transaction Amount')
axis[0].text(0.3e8, 2750000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['amount'])))

axis[1].hist(np.sqrt(transfer_or_cash_out['amount']))
axis[1].set_xlabel('Square Root of Transaction Amount')
axis[1].set_title('SQRT on Transaction Amount')
axis[1].text(3000, 2650000, 'Skewness: {:.2f}'.format(skew(np.sqrt(transfer_or_cash_out['amount']))))

axis[2].hist(transfer_or_cash_out['amount_boxcox'])
axis[2].set_xlabel('Boxcox of Transaction Amount')
axis[2].set_title('Boxcox on Transaction Amount')
axis[2].text(-2, 1625000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['amount_boxcox'])))

plt.show()

In [ ]:
figure, axis = plt.subplots(1, 3, figsize = (12, 5))

axis[0].hist(transfer_or_cash_out['oldbalanceOrg'])
axis[0].set_xlabel('Original Old Balance')
axis[0].set_title('Original Old Balance')
axis[0].text(0.2e8, 2650000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['oldbalanceOrg'])))

axis[1].hist(np.sqrt(transfer_or_cash_out['oldbalanceOrg']))
axis[1].set_xlabel('Square Root of Original Old Balance')
axis[1].set_title('SQRT on Original Old Balance')
axis[1].text(2500, 2650000, 'Skewness: {:.2f}'.format(skew(np.sqrt(transfer_or_cash_out['oldbalanceOrg']))))

axis[2].hist(transfer_or_cash_out['amount_oldbalanceOrg'])
axis[2].set_xlabel('Boxcox of Original Old Balance')
axis[2].set_title('Boxcox on Original Old Balance')
axis[2].text(0, 1275000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['amount_oldbalanceOrg'])))

plt.show()

In [ ]:
figure, axis = plt.subplots(1, 3, figsize = (12, 5))

axis[0].hist(transfer_or_cash_out['newbalanceOrig'])
axis[0].set_xlabel('Original New Balance')
axis[0].set_title('Original New Balance')
axis[0].text(0.1e8, 2650000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['newbalanceOrig'])))

axis[1].hist(np.sqrt(transfer_or_cash_out['newbalanceOrig']))
axis[1].set_xlabel('Square Root of Original New Balance')
axis[1].set_title('SQRT on Original New Balance')
axis[1].text(2250, 2600000, 'Skewness: {:.2f}'.format(skew(np.sqrt(transfer_or_cash_out['newbalanceOrig']))))

axis[2].hist(transfer_or_cash_out['amount_newbalanceOrig'])
axis[2].set_xlabel('Boxcox of Original New Balance')
axis[2].set_title('Boxcox on Original New Balance')
axis[2].text(0.75, 2300000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['amount_newbalanceOrig'])))

plt.show()

In [ ]:
figure, axis = plt.subplots(1, 3, figsize = (12, 5))

axis[0].hist(transfer_or_cash_out['oldbalanceDest'])
axis[0].set_xlabel('Destination Old Balance')
axis[0].set_title('Destination Old Balance')
axis[0].text(0.1e9, 2700000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['oldbalanceDest'])))

axis[1].hist(np.sqrt(transfer_or_cash_out['oldbalanceDest']))
axis[1].set_xlabel('Square Root of Destination Old Balance')
axis[1].set_title('SQRT on Destination Old Balance')
axis[1].text(6000, 2350000, 'Skewness: {:.2f}'.format(skew(np.sqrt(transfer_or_cash_out['oldbalanceDest']))))

axis[2].hist(transfer_or_cash_out['amount_oldbalanceDest'])
axis[2].set_xlabel('Boxcox of Destination Old Balance')
axis[2].set_title('Boxcox on Destination Old Balance')
axis[2].text(0.75, 1000000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['amount_oldbalanceDest'])))

plt.show()

In [ ]:
figure, axis = plt.subplots(1, 3, figsize = (12, 5))

axis[0].hist(transfer_or_cash_out['newbalanceDest'])
axis[0].set_xlabel('Destination New Balance')
axis[0].set_title('Destination New Balance')
axis[0].text(0.1e9, 2700000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['newbalanceDest'])))

axis[1].hist(np.sqrt(transfer_or_cash_out['newbalanceDest']))
axis[1].set_xlabel('Square Root of Destination New Balance')
axis[1].set_title('SQRT on Destination New Balance')
axis[1].text(2250, 2650000, 'Skewness: {:.2f}'.format(skew(np.sqrt(transfer_or_cash_out['newbalanceDest']))))

axis[2].hist(transfer_or_cash_out['amount_newbalanceDest'])
axis[2].set_xlabel('Boxcox of Destination New Balance')
axis[2].set_title('Boxcox on Destination New Balance')
axis[2].text(0, 1200000, 'Skewness: {:.2f}'.format(skew(transfer_or_cash_out['amount_newbalanceDest'])))

plt.show()

In [ ]:
print('Percentage of fraud transactions of the filtered dataset: {}%'.format((len(transfer_or_cash_out[transfer_or_cash_out['isFraud'] == 1]) / len(transfer_or_cash_out)) * 100))

As we can observe, there is approximately only 0.3% of the actual fraud data and the remaining unrelevant data has been filtered out.

Now, only boxcox data transformation will be used for the model prediction.

In [ ]:
transfer_or_cash_out.drop(['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest'], axis = 1, inplace = True)

In [ ]:
transfer_or_cash_out.reset_index(drop = True, inplace = True)

fraud_record_count = len(transfer_or_cash_out[transfer_or_cash_out['isFraud'] == 1])

fraud_indices = transfer_or_cash_out[transfer_or_cash_out['isFraud'] == 1].index.values
normal_indices = transfer_or_cash_out[transfer_or_cash_out['isFraud'] == 0].index

random_normal_indices = np.array(np.random.choice(normal_indices, fraud_record_count, replace = False))

combine_sample_indices = np.concatenate([fraud_indices, random_normal_indices])
combine_sample_data = transfer_or_cash_out.iloc[combine_sample_indices, :]

not_fraud_undersample = combine_sample_data.loc[:, combine_sample_data.columns != 'isFraud']
is_fraud_undersample = combine_sample_data.loc[:, combine_sample_data.columns == 'isFraud']

print('Percentage of normal transactions: ', len(combine_sample_data[combine_sample_data.isFraud == 0]) / len(combine_sample_data))
print('Percentage of fraud transactions: ', len(combine_sample_data[combine_sample_data.isFraud == 1]) / len(combine_sample_data))
print('Total count of sample transactions data: ', len(combine_sample_data))

## Model Training & Building

Train & Test Data Split

In [ ]:
X = transfer_or_cash_out.iloc[:, transfer_or_cash_out.columns != 'isFraud']
y = transfer_or_cash_out.iloc[:, transfer_or_cash_out.columns == 'isFraud']

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 5)

In [ ]:
undersample_train_x, undersample_test_x, undersample_train_y, undersample_test_y = train_test_split(not_fraud_undersample, is_fraud_undersample, test_size = 0.2, random_state = 5)

### XGBoost Model

In [ ]:
weights = sum(y.isFraud == 0) / sum(y.isFraud == 1)
xgboost = XGBClassifier(scale_pos_weight = weights, n_jobs = 8, max_depth = 3)
probability = xgboost.fit(train_x, train_y).predict_proba(test_x)
print('Area under Precision-Recall curve(AUPRC): {}'.format(metrics.average_precision_score(test_y, probability[:, 1])))

Important features for the ML model:

In [ ]:
figure = plt.figure(figsize = (14, 8))
axis = figure.add_subplot(1, 1, 1)

line_color = plt.cm.gist_rainbow(np.linspace(0, 1, 9))

axis = plot_importance(xgboost, height = 0.75, grid = False, importance_type = 'cover', ax = axis, show_values = False, color = line_color)

for area in ['top', 'bottom', 'left', 'right']:
    axis.spines[area].set_linewidth(5)

axis.set_xlabel('Relative Score', size = 15)
axis.set_ylabel('Features', size = 15)
axis.set_yticklabels(axis.get_yticklabels(), size = 12)
axis.set_title('Order of the Important Features for building the ML model', size = 18)

In [ ]:
train_sizes, train_scores, cross_val_scores = learning_curve(XGBClassifier(scale_pos_weight = weights, n_jobs = 8, max_depth = 3), train_x, train_y, scoring = 'average_precision')

In [ ]:
train_scores_mean = np.mean(train_scores, axis = 1)
cross_val_scores_mean = np.mean(cross_val_scores, axis = 1)
train_scores_std = np.std(train_scores, axis = 1)
cross_val_scores_std = np.std(cross_val_scores, axis = 1)

plt_color = plt.cm.Set2(np.linspace(0, 1, 9))

figure = plt.figure(figsize = (12, 6))

plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha = 0.25, color = plt_color[0])
plt.fill_between(train_sizes, cross_val_scores_mean - cross_val_scores_std, cross_val_scores_mean + cross_val_scores_std, alpha = 0.25, color = plt_color[1])
plt.plot(train_sizes, train_scores_mean, 'o-', label = 'train', color = plt_color[0])
plt.plot(train_sizes, cross_val_scores_mean, 'o-', label = 'cross-val', color = plt_color[1])

axis = plt.gca()

for area in ['top', 'bottom', 'left', 'right']:
    axis.spines[area].set_linewidth(5)
    
handles, labels = axis.get_legend_handles_labels()

plt.legend(handles, ['train', 'cross-val'], bbox_to_anchor = (0.25, 0.25), shadow = True)
plt.xlabel('Training Set', size = 12)
plt.ylabel('AUPRC', size = 12)
plt.title('Learning Curve Indicates A Good Accuracy Model', size = 16)

### XGBoost - Binary Classifier

In [ ]:
xgb_boost = XGBClassifier(objective = 'binary:logistic')
xgb_boost.fit(undersample_train_x, undersample_train_y.values.ravel())
xgb_boost_predict = xgb_boost.predict(undersample_test_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, xgb_boost_predict)))

In [ ]:
xgb_boost_train_predict = xgb_boost.predict(undersample_train_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_train_y, xgb_boost_train_predict)))

Check for Overfitting

In [ ]:
print('Training Set Score: {}'.format(xgb_boost.score(undersample_train_x, undersample_train_y)))
print('Testing Set Score: {}'.format(xgb_boost.score(undersample_test_x, undersample_test_y)))

Confusion Matrix

In [ ]:
xgb_regression_ConfusionMatrix = metrics.confusion_matrix(undersample_test_y, xgb_boost_predict)

print('Confusion Matrix:\n', xgb_regression_ConfusionMatrix)

print('\nTrue Positive[TP]: {}'.format(xgb_regression_ConfusionMatrix[0, 0]))
print('False Positive[FP]: {}'.format(xgb_regression_ConfusionMatrix[0, 1]))
print('False Negative[FN]: {}'.format(xgb_regression_ConfusionMatrix[1, 0]))
print('True Negative[TN]: {}'.format(xgb_regression_ConfusionMatrix[1, 1]))

In [ ]:
xgb_regression_confusion_matrix_plot = pd.DataFrame(data = xgb_regression_ConfusionMatrix, columns = ['Actual Positive', 'Actual Negative'],
                                    index = ['Predict Positive', 'Predict Negative'])
sns.heatmap(xgb_regression_confusion_matrix_plot, annot = True, cmap = 'GnBu')

Classification Metrics

In [ ]:
print(metrics.classification_report(undersample_test_y, xgb_boost_predict))

Optimal Hyperparameter Tuning for Ideal XGBoost Model

In [ ]:
parameter = {
    'colsample_bytree': uniform(0.7, 0.3),
    'gamma': uniform(0, 0.5),
    'learning_rate': uniform(0.05, 0.25),
    'max_depth': randint(2, 6),
    'n_estimators': randint(100, 150),
    'subsample': uniform(0.6, 0.4)
}

In [ ]:
random_search_cv = RandomizedSearchCV(xgb_boost, param_distributions = parameter, n_iter = 200, cv = 3, verbose = 1, n_jobs = 2, return_train_score = True)
random_search_cv.fit(undersample_test_x, undersample_test_y)

# print('Score: {}'.format(random_search_cv.cv_results_.keys()))

for i in range(1, 5):
    candidates = np.flatnonzero(random_search_cv.cv_results_['rank_test_score'] == i)
    for candidate in candidates:
        print('Model\'s Rank: {}'.format(i))
        print('Mean Validation Score: {} ; Standard Deviation Score: {}'.format(random_search_cv.cv_results_['mean_test_score'][candidate], random_search_cv.cv_results_['std_test_score'][candidate]))
        print('Parameter: {}'.format(random_search_cv.cv_results_['params'][candidate]))

In [ ]:
ideal_xgb_boost = XGBClassifier(objective = 'binary:logistic', eval_metric = ['auc'], n_estimators = 20)
ideal_xgb_boost.fit(undersample_train_x, undersample_train_y.values.ravel(), early_stopping_rounds = 5, eval_set = [(undersample_test_x, undersample_test_y)])
ideal_xgb_boost_predict = ideal_xgb_boost.predict(undersample_test_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, ideal_xgb_boost_predict)))

print('\nBest Score: {}\nBest Iteration: {}\nBest ntree limit: {}'.format(ideal_xgb_boost.best_score, ideal_xgb_boost.best_iteration, ideal_xgb_boost.best_ntree_limit))

### Random Forest Classifier Model

In [ ]:
random_forest_classifier = RandomForestClassifier(n_estimators = 100)
random_forest_classifier.fit(undersample_train_x, undersample_train_y.values.ravel())
random_forest_classifier_predict = random_forest_classifier.predict(undersample_test_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, random_forest_classifier_predict)))

In [ ]:
random_forest_classifier_train_predict = random_forest_classifier.predict(undersample_train_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_train_y, random_forest_classifier_train_predict)))

Check for Overfitting

In [ ]:
print('Training Set Score: {}'.format(random_forest_classifier.score(undersample_train_x, undersample_train_y)))
print('Testing Set Score: {}'.format(random_forest_classifier.score(undersample_test_x, undersample_test_y)))

Confusion Matrix

In [ ]:
RFC_ConfusionMatrix = metrics.confusion_matrix(undersample_test_y, random_forest_classifier_predict)

print('Confusion Matrix:\n', RFC_ConfusionMatrix)

print('\nTrue Positive[TP]: {}'.format(RFC_ConfusionMatrix[0, 0]))
print('False Positive[FP]: {}'.format(RFC_ConfusionMatrix[0, 1]))
print('False Negative[FN]: {}'.format(RFC_ConfusionMatrix[1, 0]))
print('True Negative[TN]: {}'.format(RFC_ConfusionMatrix[1, 1]))

In [ ]:
rfc_confusion_matrix_plot = pd.DataFrame(data = RFC_ConfusionMatrix, columns = ['Actual Positive', 'Actual Negative'],
                                    index = ['Predict Positive', 'Predict Negative'])
sns.heatmap(rfc_confusion_matrix_plot, annot = True, cmap = 'GnBu')

Classification Metrics

In [ ]:
print(metrics.classification_report(undersample_test_y, random_forest_classifier_predict))

### LightGBM Model

In [ ]:
lightGBM_model = lgb.LGBMClassifier()
lightGBM_model.fit(undersample_train_x, undersample_train_y.values.ravel())
lightGBM_model_predict = lightGBM_model.predict(undersample_test_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, lightGBM_model_predict)))

In [ ]:
lightGBM_model_train_predict = lightGBM_model.predict(undersample_train_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_train_y, lightGBM_model_train_predict)))

Check for Overfitting

In [ ]:
print('Training Set Score: {}'.format(lightGBM_model.score(undersample_train_x, undersample_train_y)))
print('Testing Set Score: {}'.format(lightGBM_model.score(undersample_test_x, undersample_test_y)))

Confusion Matrix

In [ ]:
LGBM_ConfusionMatrix = metrics.confusion_matrix(undersample_test_y, lightGBM_model_predict)

print('Confusion Matrix:\n', LGBM_ConfusionMatrix)

print('\nTrue Positive[TP]: {}'.format(LGBM_ConfusionMatrix[0, 0]))
print('False Positive[FP]: {}'.format(LGBM_ConfusionMatrix[0, 1]))
print('False Negative[FN]: {}'.format(LGBM_ConfusionMatrix[1, 0]))
print('True Negative[TN]: {}'.format(LGBM_ConfusionMatrix[1, 1]))

In [ ]:
lgbm_confusion_matrix_plot = pd.DataFrame(data = LGBM_ConfusionMatrix, columns = ['Actual Positive', 'Actual Negative'],
                                    index = ['Predict Positive', 'Predict Negative'])
sns.heatmap(lgbm_confusion_matrix_plot, annot = True, cmap = 'GnBu')

Classification Metrics

In [ ]:
print(metrics.classification_report(undersample_test_y, lightGBM_model_predict))

### Logistic Regression

In [ ]:
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(undersample_train_x, undersample_train_y.values.ravel())
logistic_regression_model_predict = logistic_regression_model.predict(undersample_test_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, logistic_regression_model_predict)))

In [ ]:
k_fold = StratifiedKFold(5)
cross_validation_score = cross_val_score(logistic_regression_model, undersample_train_x, undersample_train_y.values.ravel(), cv = k_fold, scoring = 'recall')

print('Average Recall Score across Validation Set: {}'.format(np.mean(cross_validation_score)))

Check for Overfitting

In [ ]:
print('Training Set Score: {}'.format(logistic_regression_model.score(undersample_train_x, undersample_train_y)))
print('Testing Set Score: {}'.format(logistic_regression_model.score(undersample_test_x, undersample_test_y)))

Confusion Matrix

In [ ]:
logistic_regression_ConfusionMatrix = metrics.confusion_matrix(undersample_test_y, logistic_regression_model_predict)

print('Confusion Matrix:\n', logistic_regression_ConfusionMatrix)

print('\nTrue Positive[TP]: {}'.format(logistic_regression_ConfusionMatrix[0, 0]))
print('False Positive[FP]: {}'.format(logistic_regression_ConfusionMatrix[0, 1]))
print('False Negative[FN]: {}'.format(logistic_regression_ConfusionMatrix[1, 0]))
print('True Negative[TN]: {}'.format(logistic_regression_ConfusionMatrix[1, 1]))

In [ ]:
logistic_regression_confusion_matrix_plot = pd.DataFrame(data = logistic_regression_ConfusionMatrix, columns = ['Actual Positive', 'Actual Negative'],
                                    index = ['Predict Positive', 'Predict Negative'])
sns.heatmap(logistic_regression_confusion_matrix_plot, annot = True, cmap = 'GnBu')

In [ ]:
print(metrics.classification_report(undersample_test_y, logistic_regression_model_predict))

Optimal Hyperparameter Tuning for Ideal Logistic Regression

In [ ]:
parameter = [
    {'penalty': ['l1', 'l2'],
     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
     'solver': ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga'],
     'max_iter': [100, 150, 500, 1000]
    }
]

In [ ]:
lr_grid_search_cv = GridSearchCV(logistic_regression_model, param_grid = parameter, cv = 3)
lr_grid_search_cv_fit = lr_grid_search_cv.fit(undersample_train_x.values, undersample_train_y.values.ravel())
lr_grid_search_cv_fit

In [ ]:
print('Best Parameter for Logistic Regression: {}'.format(lr_grid_search_cv_fit.best_params_))

In [ ]:
print('Accuracy: {}'.format(lr_grid_search_cv_fit.score(undersample_train_x.values, undersample_train_y.values)))

Area Under Curve(AUC) - Evaluation Metric

In [ ]:
ideal_logistic_regression_model = LogisticRegression(C = 100, penalty = 'l2', solver = 'lbfgs', max_iter = 500)
ideal_logistic_regression_model_fit = ideal_logistic_regression_model.fit(undersample_train_x.values, undersample_train_y.values.ravel()).decision_function(undersample_test_x.values)
ideal_logistic_regression_model_predict = ideal_logistic_regression_model.predict(undersample_test_x)

false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(undersample_test_y.values.ravel(), ideal_logistic_regression_model_fit)
roc_auc_lr = metrics.auc(false_positive_rate, true_positive_rate)
print('AUC Value: {}'.format(roc_auc_lr))

plt.title('ROC Curve', size = 15)
plt.plot(false_positive_rate, true_positive_rate, 'r', label = 'AUC: {:0.2f}'.format(roc_auc_lr))
plt.plot([0, 1], [0, 1], 'g--')
plt.legend(loc = 'lower right')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

### Decision Trees 

In [ ]:
decision_tree_classifier_model = DecisionTreeClassifier()
decision_tree_classifier_model.fit(undersample_train_x, undersample_train_y.values.ravel())
decision_tree_classifier_model_predict = decision_tree_classifier_model.predict(undersample_test_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, decision_tree_classifier_model_predict)))

In [ ]:
decision_tree_classifier_model_train_predict = decision_tree_classifier_model.predict(undersample_train_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_train_y, decision_tree_classifier_model_train_predict)))

Check for Overfitting

In [ ]:
print('Training Set Score: {}'.format(decision_tree_classifier_model.score(undersample_train_x, undersample_train_y)))
print('Testing Set Score: {}'.format(decision_tree_classifier_model.score(undersample_test_x, undersample_test_y)))

Confusion Matrix

In [ ]:
decision_tree_ConfusionMatrix = metrics.confusion_matrix(undersample_test_y, decision_tree_classifier_model_predict)

print('Confusion Matrix:\n', decision_tree_ConfusionMatrix)

print('\nTrue Positive[TP]: {}'.format(decision_tree_ConfusionMatrix[0, 0]))
print('False Positive[FP]: {}'.format(decision_tree_ConfusionMatrix[0, 1]))
print('False Negative[FN]: {}'.format(decision_tree_ConfusionMatrix[1, 0]))
print('True Negative[TN]: {}'.format(decision_tree_ConfusionMatrix[1, 1]))

In [ ]:
decision_tree_confusion_matrix_plot = pd.DataFrame(data = decision_tree_ConfusionMatrix, columns = ['Actual Positive', 'Actual Negative'],
                                    index = ['Predict Positive', 'Predict Negative'])
sns.heatmap(decision_tree_confusion_matrix_plot, annot = True, cmap = 'GnBu')

Classification Metrics

In [ ]:
print(metrics.classification_report(undersample_test_y, decision_tree_classifier_model_predict))

Area Under Curve(AUC) - Evaluation Metric

In [ ]:
false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(undersample_test_y.values.ravel(), decision_tree_classifier_model_predict)
roc_auc_decision_tree = metrics.auc(false_positive_rate, true_positive_rate)
print('AUC Value: {}'.format(roc_auc_decision_tree))

Optimal Hyperparameter Tuning for Ideal Decision Tree 

In [ ]:
parameter = [
    {'max_depth': np.linspace(1, 32, 32, endpoint = True),
     'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint = True),
     'min_samples_split': np.linspace(0.25, 1.0, 10, endpoint = True),
     'max_features': list(range(1, undersample_train_x.shape[1]))
    }
]

In [ ]:
decision_tree_grid_search_cv = GridSearchCV(decision_tree_classifier_model, param_grid = parameter, cv = 3)
decision_tree_grid_search_cv_fit = decision_tree_grid_search_cv.fit(undersample_train_x.values, undersample_train_y.values.ravel())
decision_tree_grid_search_cv_fit

In [ ]:
print('Best Parameter for Decision Tree: {}'.format(decision_tree_grid_search_cv_fit.best_params_))

In [ ]:
print('Accuracy: {}'.format(decision_tree_grid_search_cv_fit.score(undersample_train_x.values, undersample_train_y.values)))

Area Under Curve(AUC) - Evaluation Metric

In [ ]:
ideal_decision_tree_classifier_model = DecisionTreeClassifier(max_depth = 18.0, max_features = 3, min_samples_leaf = 0.1, min_samples_split = 0.3333333333333333)
ideal_decision_tree_classifier_model.fit(undersample_train_x, undersample_train_y.values.ravel())
ideal_decision_tree_classifier_model_predict = ideal_decision_tree_classifier_model.predict(undersample_test_x)

print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, ideal_decision_tree_classifier_model_predict)))

## Summary

### XGBoost Model - Binary Classifier

In [ ]:
print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, ideal_xgb_boost_predict)))

### Random Forest Classifier 

In [ ]:
print('Accuracy of Random Forest Classifier: {}'.format(metrics.accuracy_score(undersample_test_y, random_forest_classifier_predict)))

### LightGBM Model

In [ ]:
print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, lightGBM_model_predict)))

### Logistic Regression

In [ ]:
print('Accuracy : {}'.format(metrics.accuracy_score(undersample_test_y, ideal_logistic_regression_model_predict)))

### Decision Trees

In [ ]:
print('Accuracy: {}'.format(metrics.accuracy_score(undersample_test_y, decision_tree_classifier_model_predict)))

## *******************************************
### Best Model - LightGBM(99.3% accuracy)
## *******************************************